In [1]:
# import everything 
from importlib import reload
import ecc
import helper


### Create a testnet address using your own secret key
#### Use your phone number twice as a basic sample
#### Make sure to set: %env TESTNET_SECRET=123  to a meaningfully large secret



In [1]:
from ecc import G
import os
#phone number twice maybe
%env TESTNET_SECRET=123
secret = int(os.environ['TESTNET_SECRET'])
point = secret*G
print(point.address(testnet=True))

env: TESTNET_SECRET=123
miXckrsqnNrDYroEK97dhozJGuebj4ee4L


1. **Find a testnet faucet address to transfer Bitcoin to the above address. Try https://testnet.coinfaucet.eu/en/**
2. **Copy the transaction id from the faucet and use that below.**
3. **Create a BTCR DID from this address.**

In [29]:
from ecc import PrivateKey
from helper import decode_base58, p2pkh_script, SIGHASH_ALL
from script import Script
from tx import TxIn, TxOut, Tx

initial_faucet_transaction = '1687489bb20affe2c94b1057a2236daaffafd7aadb371697c916fd3e356efa1a'

secret = int(os.environ['TESTNET_SECRET'])
point = secret*G
address = point.address(testnet=True)
prev_tx = bytes.fromhex(initial_faucet_transaction)
prev_index = 1 # FILL THIS IN
target_address = address
target_amount = 0.00
change_address = address
priv = PrivateKey(secret=secret)

# initialize inputs
# create a new tx input with prev_tx, prev_index, blank script_sig and max sequence
tx_in = TxIn(
            prev_tx=prev_tx,
            prev_index=prev_index,
            script_sig=b'',
            sequence=0xffffffff,
        )
change_amount = int(tx_in.value()) - 1

# initialize outputs
tx_outs = []
h160 = decode_base58(target_address)
tx_outs.append(TxOut(
    amount=int(target_amount),
    script_pubkey=p2pkh_script(h160),
))
h160 = decode_base58(change_address)
tx_outs.append(TxOut(
    amount=int(change_amount),
    script_pubkey=p2pkh_script(h160),
))

# create the transaction
tx_obj = Tx(version=1, tx_ins=[tx_in], tx_outs=tx_outs, locktime=0, testnet=True)
# now sign the 0th input with the private key using SIGHASH_ALL using sign_input

tx_obj.sign_input(0, priv, SIGHASH_ALL)

# SANITY CHECK: change address corresponds to private key
if priv.point.address(testnet=True) != change_address:
    raise RuntimeError('Private Key does not correspond to Change Address, check priv_key and change_address')

# SANITY CHECK: output's script_pubkey is the same one as your address
if tx_in.script_pubkey(testnet=True).elements[2] != decode_base58(change_address):
    raise RuntimeError('Output is not something you can spend with this private key. Check that the prev_tx and prev_index are correct')

# SANITY CHECK: fee is reasonable
if tx_obj.fee() > 0.05*100000000 or tx_obj.fee() <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))

# serialize and hex()
print(tx_obj.serialize().hex())

#prior run resulted in this transaction.
#1687489bb20affe2c94b1057a2236daaffafd7aadb371697c916fd3e356efa1a

01000000011afa6e353efd16c9971637dbaad7afffaa6d23a257104bc9e2ff0ab29b488716010000006a473044022069712bec07b67044aee6bdc8c28eb7ced7abb0bec07d78bad2421ba836b23ee402200609abfaa4cb8163f378bedaab93b480b5742ed99ca3f67b5c9513b662e42b40012103df0a618b6b4599ef43dcef41727f5124cd2b47f76dbd5e17093ad46f7f469b99ffffffff0200000000000000001976a914ed03e3b7142101938e79fb1ab7f136f4eeeb326088ac3f805800000000001976a914ed03e3b7142101938e79fb1ab7f136f4eeeb326088ac00000000
